# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

## Esquema Relacional
#### Tabelas
* Demographic Person (__id__, gender, age);
* Medications Drug (__code__, name);
* Medications Use (__person_id__, __drug_code__, days_use)
    * CHE: person_id, Chave estrangeira para Demographic Person
    * CHE: drug_code, Chave estrangeira para Medications Drug

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Pessoa;
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../data/demographic-person.csv');

In [3]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [8]:
DROP TABLE IF EXISTS Medicamento;
CREATE TABLE Medicamento (
    Codigo VARCHAR(6) NOT NULL,
    Nome VARCHAR(100) NOT NULL,
    PRIMARY KEY(Codigo)
) AS SELECT
    code,
    name
  FROM CSVREAD('../data/medications-drug.csv');

In [9]:
SELECT * FROM Medicamento;

In [11]:
DROP TABLE IF EXISTS Uso_Medicamento;
CREATE TABLE Uso_Medicamento(
    Id_Pessoa VARCHAR(5) NOT NULL,
    Codigo_Medicamento VARCHAR(6) NOT NULL,
    Dias_Uso INTEGER,
    FOREIGN KEY(Id_Pessoa)
        REFERENCES Pessoa(Id)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY(Codigo_Medicamento)
        REFERENCES MEdicamento(Codigo)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,    
) AS SELECT
    person_id,
    drug_code,
    days_use
  FROM CSVREAD('../data/medications-use.csv');

In [12]:
SELECT * FROM Uso_Medicamento;

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [15]:
SELECT  *
        FROM Pessoa P
        WHERE P.Age > 60
        ORDER BY P.Age;

b) Liste as idades das pessoas cadastradas (sem repetições)

In [14]:
SELECT  DISTINCT P.Age
        FROM Pessoa P
        ORDER BY P.Age;

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [18]:
SELECT  P.Id, P.Age, U.Codigo_Medicamento, U.Dias_Uso
        FROM Pessoa P, Uso_MEDICAMENTO U
        WHERE P.Id = U.Id_Pessoa
        ORDER BY P.Id;

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [21]:
SELECT  P.Id, P.Age, M.Nome, U.Dias_Uso
        FROM Pessoa P, Medicamento M, Uso_Medicamento U
        WHERE P.Id = U.Id_Pessoa AND M.Codigo = U.Codigo_Medicamento;

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [23]:
SELECT  DISTINCT M.Nome
        FROM Medicamento M, Uso_Medicamento U
        WHERE M.Codigo = U.Codigo_Medicamento
        ORDER BY M.Nome;

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [26]:
SELECT  DISTINCT M.Nome
        FROM Medicamento M, Uso_Medicamento U
        WHERE M.Codigo = U.Codigo_Medicamento AND U.Dias_Uso > 2000
        ORDER BY M.Nome;